# Whittaker vectors for $\mathfrak{gl}_N$: tutorial



This is a tutorial to the `whittaker` "package" that allows to compute Whittaker vectors in the tensor product $\mathcal{W} \otimes (\mathrm{Sym}^{\bullet} \mathbb{C}^N)^{\otimes m}$ using the Kirillov projector from <a href="https://arxiv.org/abs/2310.06669">arXiv:2310.06669</a>, where $\mathcal{W} = \mathbb{C}_{\psi} \otimes_{\mathfrak{n}_-} \mathrm{U}(\mathfrak{gl}_N)$ (for notations, we refer to <em>loc. cit.</em>).

The "Whittaker" class has four parameters:
<ul>
    <li> <b>N</b> : <b style="color:#ff3399">int</b> — dimension; </li>
    <li> <b>asymptotic</b> : <b style="color:#ff3399">bool</b> — if "True", the constructions use the formal parameter $\hbar$ considered as an element of the Laurent polynomial ring (in particular, the expression $\hbar^{-1}$ is defined, but $1/(\hbar-1)$ is not); if "False", then $\hbar=1$.</li>
    <li> <b>prefixes</b> : <b style="color:#ff3399">list</b> — defines the variable names in each tensor component $\mathrm{Sym}^{\bullet} \mathbb{C}^N$.  </li>
    <li> <b>base</b> : <b style="color:#ff3399">Ring</b> - base ring for the Lie algebra (by default, it is the ring $QQ$ of rational numbers). </li>
</ul>

Let us introduce the `Whittaker` class. All the constructions use variable `t`.

In [20]:
from whittaker import Whittaker

W = Whittaker(N=4,asymptotic=True,prefixes=['u','v'])
G,t,N,h = W.G,W.t,W.N,W.hbar

The Lie algebra elements are introduced as `G['E_i_j']`. The PBW basis of $\mathrm{U}_{\hbar} (\mathfrak{gl}_N)$ is with respect to the triangular decomposition $\mathfrak{gl}_N = \mathfrak{n}_- \oplus \mathfrak{h} \oplus \mathfrak{n}_+$, where $\mathfrak{n}_-$ (resp. $\mathfrak{n}_+)$ is the space of lower (resp. upper) triangular matrices and $\mathfrak{h}$ is the diagonal ones. 

The vectors can be introduced as `G['v_i']` (`v` should be changed to the prefixes one set). In our case, the elements of copies of $\mathrm{Sym}^{\bullet} \mathbb{C}^N$ correspond to variables $\{u_i\}, \{v_j\}$. 

In [2]:
print("E_{21}*E_{12}: ",G['E_2_1']*G['E_1_2'])
print("E_{12}*E_{21}: ",G['E_1_2']*G['E_2_1'])

print("v_1: ", G['v_1'])

E_{21}*E_{12}:  G['E_2_1']*G['E_1_2']
E_{12}*E_{21}:  G['E_2_1']*G['E_1_2'] + h*G['E_1_1'] - h*G['E_2_2']
v_1:  G['v_1']


To make life easier, one can use the function `vector` which outputs the list of basis vector with the corresponding prefix.

In [3]:
v,u = W.vector('v'), W.vector('u')
v[1] == G['v_1']

True

We consider $\mathrm{U}_{\hbar}(\mathfrak{gl}_N) \otimes (\mathrm{Sym}^{\bullet} \mathbb{C}^N)^{\otimes m}$ as a free Harish-Chandra bimodule in the notations of <a href="https://arxiv.org/abs/2310.06669">arXiv:2310.06669</a>; in particular, let us see the difference between the left and the right multiplication.

In [4]:
print("E_{21}*v_1: ",G['E_2_1']*v[1])
print("v_1*E_{12}: ",v[1]*G['E_2_1'])

E_{21}*v_1:  G['E_2_1']*G['v_1']
v_1*E_{12}:  G['E_2_1']*G['v_1'] - h*G['v_2']


We identify $\mathcal{W} \otimes (\mathrm{Sym}^{\bullet} \mathbb{C}^N)^{\otimes m}$ with $\mathrm{U}_{\hbar}(\mathfrak{b}) \otimes (\mathrm{Sym}^{\bullet} \mathbb{C}^N)^{\otimes m}$, where $\mathfrak{b} = \mathfrak{h} \oplus \mathfrak{n}_+$ is the Borel subalgebra. To get the corresponding element of $\mathcal{W} \otimes (\mathrm{Sym}^{\bullet} \mathbb{C}^N)^{\otimes m}$, we apply the function `quotient`:

In [5]:
print("Class of (E_{21}*v_1): ",W.quotient(G['E_2_1']*v[1]))
print("Class of (E_{31}*E_{11}*v_1): ",W.quotient(G['E_3_1']*G['E_1_1']*v[1]))

Class of (E_{21}*v_1):  G['v_1']
Class of (E_{31}*E_{11}*v_1):  0


Before showing how the Kirillov projector works, let us introduce the main component in its definition - a quantum minor. Its arguments are lists corresponding to the rows and columns.

In [6]:
print(W.minor([1],[2]))
print(W.minor([1],[1]))
print(W.minor([1,2],[1,2]))

G['E_1_2']
t + G['E_1_1']
t^2 + (G['E_1_1'] + G['E_2_2'] - h)*t - G['E_2_1']*G['E_1_2'] + G['E_1_1']*G['E_2_2'] - h*G['E_1_1']


For instance, let us check that the Capelli determinant is central in $\mathrm{U}_{\hbar} (\mathfrak{gl}_N)$.

In [7]:
indices = list(range(1,N+1))
capelli = W.minor(indices,indices)
ifCentral = 1
for i in range(N):
    for j in range(N):
        ifCentral *= (G['E_' + str(i+1) + '_' + str(j+1)]*capelli - capelli*G['E_' + str(i+1) + '_' + str(j+1)] == 0)

print(ifCentral == 1)

True


The action of the series $P_{ij}^{\psi}(-u)$ of (4.1) can be compute using the `noe` function. There is a parameter `shift` corresponding to $u$. 

In [8]:
print(W.noe(2,1,v[1]))
print(W.noe(2,1,v[1],shift=2))

-G['E_1_1']*G['v_2'] + G['v_1']
-G['E_1_1']*G['v_2'] + G['v_1'] - 2*h*G['v_2']


Finally, the Kirillov projector $P_{\mathfrak{m}_N}^{\psi}(u_1,\ldots,u_{N-1})$ is given by the function `proj`. The parameters are specified by `u_1,u_2,...,u_{N-1}`.

In [9]:
vec=v[3]
print(W.proj(vec))
print(W.proj(vec,u_1=2,u_2=3))

-G['E_1_1']*G['v_4'] - G['E_2_2']*G['v_4'] - G['E_3_3']*G['v_4'] + G['v_3']
-G['E_1_1']*G['v_4'] - G['E_2_2']*G['v_4'] - G['E_3_3']*G['v_4'] + G['v_3'] - 3*h*G['v_4']


To check if the vector we obtained is Whittaker we can use the function `isInv`.

In [10]:
vec = v[1]
print("The vector is: ",vec)
print("Invariant after projector: ",W.isInv(W.proj(vec)),"\n")
print("Invariant before projector: ",W.isInv(vec))

The vector is:  G['v_1']
Invariant after projector:  True 

E_2_1:  -h*G['v_2']
E_3_1:  -h*G['v_3']
E_4_1:  -h*G['v_4']
Invariant before projector:  False


As we see, it outputs the elements under which the corresponding vector is not invariant.

Let us try something more complicated, namely, an element in the tensor product.

In [36]:
w = W.proj(v[1]*u[1])
W.isInv(w)

True

There is an additional parameter `truncation` which applies the projector $P_{\mathfrak{m}_{N-i}}^{\psi}$ for $i \geq 0$, where `i` is the truncation parameter.

In [12]:
a = v[1]
b = W.proj(a,truncation=1)
print(b)

G['v_1']


Many properties in loc. cit. were obtained with the help of the software. For instance, let us verify Lemma 4.9.

In [28]:
x=5

c = b
c = W.noe(2,1,c,shift=x)
c = W.noe(3,1,c,shift=x)

lhs = W.quotient((t-h*x)*c*W.minor([4,2],[1,2]))
rhs = W.quotient(c*G['E_4_1']*(W.minor([1,2],[1,2]) - W.minor([1,2],[1,2])(h*x)))

lhs == rhs

True

And with the character:

In [35]:
x = 4

c = b
c = W.noe(2,1,c,shift=x)
c = W.noe(3,1,c,shift=x)

lhs = W.quotient((t-h*x)*c*(W.minor([4,2,3],[1,2,3])-1))
rhs = W.quotient(c*G['E_4_1']*(W.minor([1,2,3],[1,2,3]) - W.minor([1,2,3],[1,2,3])(h*x)))

lhs == rhs

True

One can similarly verify other properties from the paper.